In [1]:
import os
import sys
import pandas as pd
from google.colab import drive

In [2]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
ROOT_DIR = '/content/gdrive/MyDrive/Colab Notebooks'
DATA_DIR = os.path.join(ROOT_DIR, 'data', 'disaster_tweets')
os.listdir(DATA_DIR)

['train.csv',
 'test.csv',
 'model',
 'old_test_train_overlap',
 'old_test_train_split',
 'submission_20210319075718.csv',
 'submission_20210319081403.csv',
 'test_formatted.csv',
 'valid_formatted.csv',
 'train_formatted.csv',
 'submission_20210319083313.csv',
 'bert-base-uncased_4']

In [4]:
os.listdir(ROOT_DIR)

['data',
 'RLIntro.ipynb',
 'Custom text classification (Kaggle)',
 'requirements.txt',
 'requirementscl.txt',
 'tmp_run.py',
 'TF Text Classification 2.ipynb',
 'TF Text Classification.ipynb']

In [5]:
if ROOT_DIR not in sys.path:
  sys.path.append(ROOT_DIR)

In [6]:
!pip install -r "$ROOT_DIR/requirementscl.txt"

     |████████████████████████████████| 194kB 23.8MB/s 
     |████████████████████████████████| 1.2MB 45.6MB/s 
     |████████████████████████████████| 2.0MB 46.8MB/s 
     |████████████████████████████████| 245kB 56.6MB/s 
     |████████████████████████████████| 112kB 55.5MB/s 
     |████████████████████████████████| 3.2MB 50.9MB/s 
     |████████████████████████████████| 890kB 45.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=00a2c3ca53c82d2c856ef0431f25721720ac8eb87379bf600c4bdb0b0d2a4af9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [7]:
import sys
model_name = "roberta-base"
num_epochs = "4"

args = ['run_tf_text_classification.py']
args.extend([
  "--train_file", os.path.join(DATA_DIR, "train_formatted.csv"),
  "--test_file", os.path.join(DATA_DIR, "test_formatted.csv"),
  "--dev_file", os.path.join(DATA_DIR, "valid_formatted.csv"),
  "--label_column_id", "0",
  "--model_name_or_path", model_name, 
  "--output_dir", os.path.join(DATA_DIR, f"{model_name}_{num_epochs}"), 
  "--num_train_epochs", num_epochs,
  "--per_device_train_batch_size", "16",
  "--per_device_eval_batch_size", "32",
  "--do_train", 
  "--do_eval",
  "--do_predict", 
  "--logging_steps", "460", ### 460 for test 256, 444 for test 512, 476 for all records in training
  "--evaluation_strategy", "steps", 
  "--save_steps", "460", ### WAS: "476", ### 2 times per epoch (476 steps per epoch)
  "--overwrite_output_dir", 
  "--max_seq_length", "128"    
])

print(args)

['run_tf_text_classification.py', '--train_file', '/content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/train_formatted.csv', '--test_file', '/content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/test_formatted.csv', '--dev_file', '/content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/valid_formatted.csv', '--label_column_id', '0', '--model_name_or_path', 'roberta-base', '--output_dir', '/content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/roberta-base_4', '--num_train_epochs', '4', '--per_device_train_batch_size', '16', '--per_device_eval_batch_size', '32', '--do_train', '--do_eval', '--do_predict', '--logging_steps', '460', '--evaluation_strategy', 'steps', '--save_steps', '460', '--overwrite_output_dir', '--max_seq_length', '128']


In [8]:
#!/usr/bin/env python
# coding=utf-8
# Copyright 2020 The HuggingFace Team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Fine-tuning the library models for sequence classification."""
import logging
import os
from dataclasses import dataclass, field
from typing import Dict, Optional

import datasets
import numpy as np
import tensorflow as tf

from transformers import (
    AutoConfig,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    PreTrainedTokenizer,
    TFAutoModelForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
)
from transformers.utils import logging as hf_logging


hf_logging.set_verbosity_info()
hf_logging.enable_default_handler()
hf_logging.enable_explicit_format()


def get_tfds(
    train_file: str,
    eval_file: str,
    test_file: str,
    tokenizer: PreTrainedTokenizer,
    label_column_id: int,
    max_seq_length: Optional[int] = None,
):
    files = {}

    if train_file is not None:
        files[datasets.Split.TRAIN] = [train_file]
    if eval_file is not None:
        files[datasets.Split.VALIDATION] = [eval_file]
    if test_file is not None:
        files[datasets.Split.TEST] = [test_file]

    ds = datasets.load_dataset("csv", data_files=files)
    features_name = list(ds[list(files.keys())[0]].features.keys())
    print(f"Features Name: {features_name}")
    
    label_name = features_name.pop(label_column_id)
    label_list = list(set(ds[list(files.keys())[0]][label_name]))
    label2id = {label: i for i, label in enumerate(label_list)}
    input_names = tokenizer.model_input_names
    transformed_ds = {}

    if len(features_name) == 1:
        for k in files.keys():
            transformed_ds[k] = ds[k].map(
                lambda example: tokenizer.batch_encode_plus(
                    example[features_name[0]], truncation=True, max_length=max_seq_length, padding="max_length"
                ),
                batched=True,
            )
    elif len(features_name) == 2:
        for k in files.keys():
            transformed_ds[k] = ds[k].map(
                lambda example: tokenizer.batch_encode_plus(
                    (example[features_name[0]], example[features_name[1]]),
                    truncation=True,
                    max_length=max_seq_length,
                    padding="max_length",
                ),
                batched=True,
            )

    def gen_train():
        for ex in transformed_ds[datasets.Split.TRAIN]:
            d = {k: v for k, v in ex.items() if k in input_names}
            label = label2id[ex[label_name]]
            yield (d, label)

    def gen_val():
        for ex in transformed_ds[datasets.Split.VALIDATION]:
            d = {k: v for k, v in ex.items() if k in input_names}
            label = label2id[ex[label_name]]
            yield (d, label)

    def gen_test():
        for ex in transformed_ds[datasets.Split.TEST]:
            d = {k: v for k, v in ex.items() if k in input_names}
            label = label2id[ex[label_name]]
            yield (d, label)

    train_ds = (
        tf.data.Dataset.from_generator(
            gen_train,
            ({k: tf.int32 for k in input_names}, tf.int64),
            ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
        )
        if datasets.Split.TRAIN in transformed_ds
        else None
    )

    if train_ds is not None:
        train_ds = train_ds.apply(tf.data.experimental.assert_cardinality(len(ds[datasets.Split.TRAIN])))

    val_ds = (
        tf.data.Dataset.from_generator(
            gen_val,
            ({k: tf.int32 for k in input_names}, tf.int64),
            ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
        )
        if datasets.Split.VALIDATION in transformed_ds
        else None
    )

    if val_ds is not None:
        val_ds = val_ds.apply(tf.data.experimental.assert_cardinality(len(ds[datasets.Split.VALIDATION])))

    test_ds = (
        tf.data.Dataset.from_generator(
            gen_test,
            ({k: tf.int32 for k in input_names}, tf.int64),
            ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
        )
        if datasets.Split.TEST in transformed_ds
        else None
    )

    if test_ds is not None:
        test_ds = test_ds.apply(tf.data.experimental.assert_cardinality(len(ds[datasets.Split.TEST])))

    return train_ds, val_ds, test_ds, label2id


logger = logging.getLogger(__name__)


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.

    Using `HfArgumentParser` we can turn this class
    into argparse arguments to be able to specify them on
    the command line.
    """

    label_column_id: int = field(metadata={"help": "Which column contains the label"})
    train_file: str = field(default=None, metadata={"help": "The path of the training file"})
    dev_file: Optional[str] = field(default=None, metadata={"help": "The path of the development file"})
    test_file: Optional[str] = field(default=None, metadata={"help": "The path of the test file"})
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    use_fast: bool = field(default=False, metadata={"help": "Set this flag to use fast tokenization."})
    # If you want to tweak more attributes on your tokenizer, you should do it in a distinct script,
    # or just modify its tokenizer_config.json.
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )

In [9]:
sys.argv = args

# See all possible arguments in src/transformers/training_args.py
# or by passing the --help flag to this script.
# We now keep distinct sets of args, for a cleaner separation of concerns.
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TFTrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses()

if (
    os.path.exists(training_args.output_dir)
    and os.listdir(training_args.output_dir)
    and training_args.do_train
    and not training_args.overwrite_output_dir
):
    raise ValueError(
        f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
    )

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(
    "n_replicas: %s, distributed training: %s, 16-bits training: %s",
    training_args.n_replicas,
    bool(training_args.n_replicas > 1),
    training_args.fp16,
)
logger.info("Training/evaluation parameters %s", training_args)


[INFO|training_args.py:631] 2021-03-19 10:09:02,513 >> PyTorch: setting up devices
[INFO|training_args.py:555] 2021-03-19 10:09:02,578 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[INFO|training_args_tf.py:192] 2021-03-19 10:09:02,583 >> Tensorflow: setting up strategy
03/19/2021 10:09:07 - INFO - __main__ -   n_replicas: 1, distributed training: False, 16-bits training: False
03/19/2021 10:09:07 - INFO - __main__ -   Training/evaluation parameters TFTrainingArguments(output_dir='/content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/roberta-base_4', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_s

In [10]:
# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.

tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
)


03/19/2021 10:09:07 - INFO - filelock -   Lock 140034830677968 acquired on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
[INFO|file_utils.py:1386] 2021-03-19 10:09:07,620 >> https://huggingface.co/roberta-base/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpd0skmiuw


[INFO|file_utils.py:1390] 2021-03-19 10:09:07,672 >> storing https://huggingface.co/roberta-base/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
[INFO|file_utils.py:1393] 2021-03-19 10:09:07,673 >> creating metadata file for /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
03/19/2021 10:09:07 - INFO - filelock -   Lock 140034830677968 released on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
[INFO|configuration_utils.py:463] 2021-03-19 10:09:07,676 >> loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/7

[INFO|file_utils.py:1390] 2021-03-19 10:09:07,886 >> storing https://huggingface.co/roberta-base/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
[INFO|file_utils.py:1393] 2021-03-19 10:09:07,887 >> creating metadata file for /root/.cache/huggingface/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
03/19/2021 10:09:07 - INFO - filelock -   Lock 140034788403472 released on /root/.cache/huggingface/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab.lock
03/19/2021 10:09:07 - INFO - filelock -   Lock 140032333103056 acquired on /root/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c80384126

[INFO|file_utils.py:1390] 2021-03-19 10:09:07,998 >> storing https://huggingface.co/roberta-base/resolve/main/merges.txt in cache at /root/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
[INFO|file_utils.py:1393] 2021-03-19 10:09:07,999 >> creating metadata file for /root/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
03/19/2021 10:09:08 - INFO - filelock -   Lock 140032333103056 released on /root/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
03/19/2021 10:09:08 - INFO - filelock -   Lock 140034788590992 acquired on /root/.cache/huggingface/transformers/d53fc0fa09b8342651efd4073d75e19617b3e51287c2a535becda5808a8db287.fc9576039592f026ad76a1c23

[INFO|file_utils.py:1390] 2021-03-19 10:09:08,149 >> storing https://huggingface.co/roberta-base/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/d53fc0fa09b8342651efd4073d75e19617b3e51287c2a535becda5808a8db287.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
[INFO|file_utils.py:1393] 2021-03-19 10:09:08,158 >> creating metadata file for /root/.cache/huggingface/transformers/d53fc0fa09b8342651efd4073d75e19617b3e51287c2a535becda5808a8db287.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
03/19/2021 10:09:08 - INFO - filelock -   Lock 140034788590992 released on /root/.cache/huggingface/transformers/d53fc0fa09b8342651efd4073d75e19617b3e51287c2a535becda5808a8db287.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730.lock


[INFO|tokenization_utils_base.py:1702] 2021-03-19 10:09:08,218 >> loading file https://huggingface.co/roberta-base/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
[INFO|tokenization_utils_base.py:1702] 2021-03-19 10:09:08,219 >> loading file https://huggingface.co/roberta-base/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
[INFO|tokenization_utils_base.py:1702] 2021-03-19 10:09:08,219 >> loading file https://huggingface.co/roberta-base/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/d53fc0fa09b8342651efd4073d75e19617b3e51287c2a535becda5808a8db287.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
[INFO|tokenization_utils_base.py:

In [11]:

train_dataset, eval_dataset, test_ds, label2id = get_tfds(
    train_file=data_args.train_file,
    eval_file=data_args.dev_file,
    test_file=data_args.test_file,
    tokenizer=tokenizer,
    label_column_id=data_args.label_column_id,
    max_seq_length=data_args.max_seq_length,
)


03/19/2021 10:09:08 - WARNING - datasets.builder -   Using custom data configuration default-b765ceac4552832d


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b765ceac4552832d/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.
Features Name: ['target', 'text']


In [12]:
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    num_labels=len(label2id),
    label2id=label2id,
    id2label={id: label for label, id in label2id.items()},
    finetuning_task="text-classification",
    cache_dir=model_args.cache_dir,
)


[INFO|configuration_utils.py:463] 2021-03-19 10:09:11,386 >> loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
[INFO|configuration_utils.py:499] 2021-03-19 10:09:11,387 >> Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "finetuning_task": "text-classification",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": 0,
    "1": 1
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0": 0,
    "1": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  

In [13]:
with training_args.strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained(
        model_args.model_name_or_path,
        from_pt=bool(".bin" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
    )



03/19/2021 10:09:11 - INFO - filelock -   Lock 140030738270096 acquired on /root/.cache/huggingface/transformers/22fef2e3c5012c1a8f8d7f024e30275dd2925b076abb5131dc3d1068345b6426.d409db346b0c1408865b9785d36744ccb988186626309ae8f995f86511811602.h5.lock
[INFO|file_utils.py:1386] 2021-03-19 10:09:11,418 >> https://huggingface.co/roberta-base/resolve/main/tf_model.h5 not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpwtlfz36v


[INFO|file_utils.py:1390] 2021-03-19 10:09:22,640 >> storing https://huggingface.co/roberta-base/resolve/main/tf_model.h5 in cache at /root/.cache/huggingface/transformers/22fef2e3c5012c1a8f8d7f024e30275dd2925b076abb5131dc3d1068345b6426.d409db346b0c1408865b9785d36744ccb988186626309ae8f995f86511811602.h5
[INFO|file_utils.py:1393] 2021-03-19 10:09:22,641 >> creating metadata file for /root/.cache/huggingface/transformers/22fef2e3c5012c1a8f8d7f024e30275dd2925b076abb5131dc3d1068345b6426.d409db346b0c1408865b9785d36744ccb988186626309ae8f995f86511811602.h5
03/19/2021 10:09:22 - INFO - filelock -   Lock 140030738270096 released on /root/.cache/huggingface/transformers/22fef2e3c5012c1a8f8d7f024e30275dd2925b076abb5131dc3d1068345b6426.d409db346b0c1408865b9785d36744ccb988186626309ae8f995f86511811602.h5.lock
[INFO|modeling_tf_utils.py:1240] 2021-03-19 10:09:22,644 >> loading weights file https://huggingface.co/roberta-base/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers

[WARNING|modeling_tf_utils.py:1298] 2021-03-19 10:09:26,396 >> All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

[WARNING|modeling_tf_utils.py:1302] 2021-03-19 10:09:26,397 >> Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def compute_metrics(p: EvalPrediction) -> Dict:
    preds = np.argmax(p.predictions, axis=1)

    return {"acc": (preds == p.label_ids).mean()}

# Initialize our Trainer
trainer = TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)



[INFO|trainer_tf.py:117] 2021-03-19 10:09:26,423 >> You are instantiating a Trainer but W&B is not installed. To use wandb logging, run `pip install wandb; wandb login` see https://docs.wandb.com/huggingface.
[INFO|trainer_tf.py:125] 2021-03-19 10:09:26,423 >> To use comet_ml logging, run `pip/conda install comet_ml` see https://www.comet.ml/docs/python-sdk/huggingface/


In [15]:
print(f"train: {len(train_dataset)}")
print(f"valid: {len(eval_dataset)}")
print(f"test: {len(test_ds)}")

train: 7357
valid: 256
test: 3263


In [16]:
# Training
if training_args.do_train:
    trainer.train()
    trainer.save_model()
    tokenizer.save_pretrained(training_args.output_dir)


[INFO|trainer_tf.py:528] 2021-03-19 10:09:26,529 >> ***** Running training *****
[INFO|trainer_tf.py:529] 2021-03-19 10:09:26,529 >>   Num examples = 7357
[INFO|trainer_tf.py:531] 2021-03-19 10:09:26,530 >>   Num Epochs = 4
[INFO|trainer_tf.py:532] 2021-03-19 10:09:26,535 >>   Instantaneous batch size per device = 16
[INFO|trainer_tf.py:534] 2021-03-19 10:09:26,542 >>   Total train batch size (w. parallel, distributed & accumulation) = 16
[INFO|trainer_tf.py:536] 2021-03-19 10:09:26,543 >>   Gradient Accumulation steps = 1
[INFO|trainer_tf.py:537] 2021-03-19 10:09:26,545 >>   Steps per epoch = 460
[INFO|trainer_tf.py:538] 2021-03-19 10:09:26,547 >>   Total optimization steps = 1840


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


03/19/2021 10:09:32 - WARNING - tensorflow -   AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x7f5c64db1ec0>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


03/19/2021 10:09:31 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
03/19/2021 10:09:33 - WARNING - tensorflow -   AutoGraph could not transform <function wrap at 0x7f5c80663c20> and will run it as-is.
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

03/19/2021 10:09:53 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


03/19/2021 10:10:18 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


03/19/2021 10:10:18 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
[INFO|trainer_tf.py:306] 2021-03-19 10:13:24,034 >> ***** Running Evaluation *****
[INFO|trainer_tf.py:307] 2021-03-19 10:13:24,036 >>   Num examples in dataset = 256
[INFO|trainer_tf.py:309] 2021-03-19 10:13:24,036 >>   Num examples in used in evaluation = 256
[INFO|trainer_tf.py:310] 2021-03-19 10:13:24,037 >>   Batch size = 32


03/19/2021 10:13:24 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


03/19/2021 10:13:24 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
[INFO|trainer_tf.py:404] 2021-03-19 10:13:27,594 >> {'eval_loss': 0.5798395276069641, 'eval_acc': 0.765625, 'epoch': 1.0, 'step': 460}
[INFO|trainer_tf.py:404] 2021-03-19 10:13:27,603 >> {'loss': 0.48558548, 'learning_rate': 3.7499995e-05, 'epoch': 1.0, 'step': 460}
[INFO|trainer_tf.py:595] 2021-03-19 10:13:38,729 >> Saving checkpoint for step 460 at /content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/roberta-base_4/checkpoint/ckpt-1
[INFO|trainer_tf.py:306] 2021-03-19 10:16:47,783 >> ***** Running Evaluation *****
[INFO|trainer_tf.py:307] 2021-03-19 10:16:47,787 >>   Num examples in dataset = 256
[INFO|trainer_tf.py:309] 2021-03-19 10:16:47,790 >>   Num examples in used in evaluation = 256
[INFO|trainer_tf.py:310] 2021-03-19 10:16:47,791 >>   Batch size = 32
[INFO|trainer_tf.py:404] 2021-03-19 10:16:49,854 >> {'eval_loss': 0.4696185290813446

In [17]:
# Evaluation
results = {}
if training_args.do_eval:
    logger.info("*** Evaluate ***")
    result = trainer.evaluate()
    output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")

    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results *****")

        for key, value in result.items():
            logger.info("  %s = %s", key, value)
            writer.write("%s = %s\n" % (key, value))

        results.update(result)


03/19/2021 10:23:56 - INFO - __main__ -   *** Evaluate ***
[INFO|trainer_tf.py:306] 2021-03-19 10:23:56,788 >> ***** Running Evaluation *****
[INFO|trainer_tf.py:307] 2021-03-19 10:23:56,789 >>   Num examples in dataset = 256
[INFO|trainer_tf.py:309] 2021-03-19 10:23:56,790 >>   Num examples in used in evaluation = 256
[INFO|trainer_tf.py:310] 2021-03-19 10:23:56,792 >>   Batch size = 32


03/19/2021 10:23:56 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


03/19/2021 10:23:56 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
[INFO|trainer_tf.py:404] 2021-03-19 10:24:00,794 >> {'eval_loss': 0.47562873363494873, 'eval_acc': 0.82421875, 'epoch': 4.0, 'step': 1840}
03/19/2021 10:24:00 - INFO - __main__ -   ***** Eval results *****
03/19/2021 10:24:00 - INFO - __main__ -     eval_loss = 0.47562873363494873
03/19/2021 10:24:00 - INFO - __main__ -     eval_acc = 0.82421875


In [18]:
results

{'eval_acc': 0.82421875, 'eval_loss': 0.47562873363494873}

In [19]:
print(label2id)

{0: 0, 1: 1}


In [20]:
# Prediction
if training_args.do_predict:
    logger.info("*** predictions ***")
    preds = trainer.predict(test_ds)

    logger.info("*** RESUTS: ***")
    logger.info(preds)
    logger.info("*** :RESUTS ***")


03/19/2021 10:24:00 - INFO - __main__ -   *** predictions ***
[INFO|trainer_tf.py:306] 2021-03-19 10:24:00,857 >> ***** Running Prediction *****
[INFO|trainer_tf.py:307] 2021-03-19 10:24:00,859 >>   Num examples in dataset = 3263
[INFO|trainer_tf.py:310] 2021-03-19 10:24:00,861 >>   Batch size = 32


03/19/2021 10:24:26 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


03/19/2021 10:24:26 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
03/19/2021 10:24:27 - INFO - __main__ -   *** RESUTS: ***
03/19/2021 10:24:27 - INFO - __main__ -   PredictionOutput(predictions=array([[-2.0568695,  2.731622 ],
       [-1.7943337,  2.4179747],
       [-2.1183205,  2.8025274],
       ...,
       [-2.165437 ,  2.8545594],
       [-1.9152114,  2.5648017],
       [-1.3553526,  1.8698064]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 0]), metrics={'eval_loss': 1.5628674077052696, 'eval_acc': 0.596077229543365})
03/19/2021 10:24:27 - INFO - __main__ -   *** :RESUTS ***


In [21]:
decoded_preds = np.argmax(preds.predictions, axis=1)

In [22]:
decoded_preds[10:30]

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1])

In [23]:
org_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [24]:
org_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [25]:
org_test['target'] = decoded_preds

In [26]:
org_test.head()

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1


In [27]:
from datetime import datetime

In [28]:
timestamp=str(datetime.now()).replace(' ','').replace(':','').replace('-','').split('.')[0]

In [29]:
org_test[['id', 'target']].to_csv(os.path.join(DATA_DIR, f'./submission_{timestamp}.csv'), index=False)